In [6]:
library(Ragas)
library(dplyr)
library(tidyr)
library(ggplot2)
library(ggrepel)
library(openxlsx)
library(wesanderson)

In [2]:
merged <- readRDS('/data/niecr/cheongj/misc/results_seurat/seurat_obj/misc_merged_hspc.rds')
merged

An object of class Seurat 
37471 features across 16220 samples within 2 assays 
Active assay: RNA (36601 features, 2000 variable features)
 22 layers present: data, counts, scale.data.1, scale.data.2, scale.data.3, scale.data.4, scale.data.5, scale.data.6, scale.data.7, scale.data.8, scale.data.9, scale.data.11, scale.data.12, scale.data.13, scale.data.14, scale.data.15, scale.data.16, scale.data.17, scale.data.18, scale.data.19, scale.data.20, scale.data
 1 other assay present: chromvar
 9 dimensional reductions calculated: pca, integrated.rpca, umap, harmony, harmony.umap, harmony2, harmony.umap2, harmony3, harmony.umap3

# edit rds metadata

In [3]:
unique(subset_merged@meta.data$main_cluster_annotation)

[1] HSPC
Levels: T B CD14 mono HSPC NK CD16 mono DC pDC Plasma

In [5]:
merged@meta.data <- merged@meta.data %>%
  mutate(Disease.Status3 = gsub("[-_ ]", ".", Disease.Status3))
merged@meta.data <- merged@meta.data %>%
  mutate(sample.id = gsub("[-_ ]", ".", sample.id))
merged@meta.data <- merged@meta.data %>%
  mutate(sample.id2 = paste(Disease.Status3, sample.id, sep = "_"))
unique(merged@meta.data$sample.id2)

[1] "Baseline.MIS.C_71010.039"          "D28.MIS.C_71022.005"              
 [3] "D28.MIS.C_71010.049"               "MISC.recovered_71003.024"         
 [5] "D28.MIS.C_71017.008"               "MISC.recovered_71022.004"         
 [7] "M6.POSSIBLE.MIS.C_71300.002"       "Baseline.MIS.C_71028.011"         
 [9] "Baseline.MIS.C_71003.024"          "D28.MIS.C_71028.011"              
[11] "MISC.recovered_71017.008"          "MISC.recovered_71022.005"         
[13] "D28.MIS.C_71010.039"               "Baseline.POSSIBLE.MIS.C_71300.002"
[15] "MISC.recovered_71010.049"          "Baseline.MIS.C_71022.004"         
[17] "D28.MIS.C_71003.024"               "D28.MIS.C_71022.004"              
[19] "MISC.recovered_71010.039"          "Baseline.MIS.C_71022.005"         
[21] "Baseline.MIS.C_71017.008"          "Baseline.MIS.C_71010.049"         
[23] "MISC.recovered_71028.011"          "D28.POSSIBLE.MIS.C_71300.002"     
[25] "_"                                 "Baseline.MIS.C_71022.002"         
[27] "D28.COVID_71010.009"               "D28.COVID_71010.057"              
[29] "MISC.recovered_71026.001"          "D28.POSSIBLE.MIS.C_71017.010"     
[31] "MISC.recovered_71010.054"          "Baseline.COVID_71010.013"         
[33] "MISC.recovered_71010.056"          "MISC.recovered_71022.002"         
[35] "Baseline.COVID_71010.009"          "M6.COVID_71010.057"               
[37] "M12.COVID_71010.013"               "D28.MIS.C_71022.002"              
[39] "M12.COVID_71010.009"               "D28.MIS.C_71026.001"              
[41] "M6.POSSIBLE.MIS.C_71017.010"       "D28.MIS.C_71010.054"              
[43] "D28.MIS.C_71010.056"               "D28.MIS.C_71022.001"              
[45] "D28.COVID_71010.019"               "M6.POSSIBLE.MIS.C_71033.001"      
[47] "Baseline.MIS.C_71017.007"          "Baseline.COVID_71300.005"         
[49] "D28.MIS.C_71010.027"               "D28.POSSIBLE.MIS.C_71028.004"     
[51] "Baseline.MIS.C_71022.001"          "MISC.recovered_71017.007"         
[53] "D28.POSSIBLE.MIS.C_71033.001"      "M12.COVID_71300.005"              
[55] "MISC.recovered_71010.029"          "Baseline.MIS.C_71010.027"         
[57] "Baseline.POSSIBLE.MIS.C_71028.004" "Baseline.COVID_71010.019"         
[59] "M12.COVID_71010.019"               "MISC.recovered_71022.001"         
[61] "D28.MIS.C_71017.007"               "D28.COVID_71300.005"              
[63] "M12.POSSIBLE.MIS.C_71028.004"      "Baseline.POSSIBLE.MIS.C_71033.001"
[65] "D28.MIS.C_71010.029"               "MISC.recovered_71010.027"

# Function to subset rds

In [14]:
subset_rds <- function(data=merged, disease_statuses, min_count = 2) {
  # Filter the data based on Disease.Status3
  filtered_data <- subset(data, subset = Disease.Status3 %in% disease_statuses)
  
  # Compute clusters to remove based on count
  clusters_to_remove <- filtered_data@meta.data %>%
    group_by(main_cluster_annotation, Disease.Status3, sample.id2) %>%
    summarize(count = n(), .groups = 'drop') %>%
    filter(count <= min_count) %>%
    pull(main_cluster_annotation) %>%
    unique()
  
  # Compute clusters to keep
  clusters_to_keep <- setdiff(unique(filtered_data@meta.data$main_cluster_annotation), clusters_to_remove)
  
  # Subset the original data to keep only the relevant clusters
  filtered_data <- subset(filtered_data, subset = main_cluster_annotation %in% clusters_to_keep)
  
  return(filtered_data)
}
#subset_merged <- subset_rds(merged,c('Baseline.COVID', 'MISC.recovered'))

# Run pseudobulk

In [6]:
subset_merged <- subset(merged, subset = Disease.Status3 %in% c('Baseline.MIS.C', 'MISC.recovered'))
Baseline_MISC_deg <- RunPseudobulkAnalysis(object = subset_merged,
                                    ident.var = "main_cluster_annotation",
                                    group.var = "Disease.Status3",
                                    sample.var = "sample.id2",
                                    group.1 = "MISC.recovered",
                                    group.2 = "Baseline.MIS.C")

Checking validity of names...

Converting Disease.Status3 to factor

Converting sample.id2 to factor

Contrast created: group_idMISC.recovered-group_idBaseline.MIS.C

Creating single cell experiment object...

Aggregating count data...

Running differential state analysis...



  |======================================================================| 100%




Formatting output...

Pseudobulk-analysis completed!



In [7]:
subset_merged <- subset(merged, subset = Disease.Status3 %in% c('D28.MIS.C', 'MISC.recovered'))
D28_MISC_deg <- RunPseudobulkAnalysis(object = subset_merged,
                                    ident.var = "main_cluster_annotation",
                                    group.var = "Disease.Status3",
                                    sample.var = "sample.id2",
                                    group.1 = "MISC.recovered",
                                    group.2 = "D28.MIS.C")

Checking validity of names...

Converting Disease.Status3 to factor

Converting sample.id2 to factor

Contrast created: group_idMISC.recovered-group_idD28.MIS.C

Creating single cell experiment object...

Aggregating count data...

Running differential state analysis...



  |======================================================================| 100%




Formatting output...

Pseudobulk-analysis completed!



In [8]:
subset_merged <- subset(merged, subset = Disease.Status3 %in% c('Baseline.COVID', 'MISC.recovered'))
#subset_merged <- subset_rds(merged,c('Baseline.COVID', 'MISC.recovered'))
Baseline_COVID_deg <- RunPseudobulkAnalysis(object = subset_merged,
                                    ident.var = "main_cluster_annotation",
                                    group.var = "Disease.Status3",
                                    sample.var = "sample.id2",
                                    group.1 = "MISC.recovered",
                                    group.2 = "Baseline.COVID")

Checking validity of names...

Converting Disease.Status3 to factor

Converting sample.id2 to factor

Contrast created: group_idMISC.recovered-group_idBaseline.COVID

Creating single cell experiment object...

Aggregating count data...

Running differential state analysis...



  |======================================================================| 100%




Formatting output...

Pseudobulk-analysis completed!



In [9]:
subset_merged <- subset(merged, subset = Disease.Status3 %in% c('D28.COVID', 'MISC.recovered'))
D28_COVID_deg <- RunPseudobulkAnalysis(object = subset_merged,
                                    ident.var = "main_cluster_annotation",
                                    group.var = "Disease.Status3",
                                    sample.var = "sample.id2",
                                    group.1 = "MISC.recovered",
                                    group.2 = "D28.COVID")

Checking validity of names...

Converting Disease.Status3 to factor

Converting sample.id2 to factor

Contrast created: group_idMISC.recovered-group_idD28.COVID

Creating single cell experiment object...

Aggregating count data...

Running differential state analysis...



  |======================================================================| 100%




Formatting output...

Pseudobulk-analysis completed!



In [16]:
subset_merged <- subset_rds(merged,c('M6.COVID', 'MISC.recovered'),min_count=1)
M6_COVID_deg <- RunPseudobulkAnalysis(object = subset_merged,
                                    ident.var = "main_cluster_annotation",
                                    group.var = "Disease.Status3",
                                    sample.var = "sample.id2",
                                    group.1 = "MISC.recovered",
                                    group.2 = "M6.COVID")

Checking validity of names...

Converting Disease.Status3 to factor

Converting sample.id2 to factor

Contrast created: group_idMISC.recovered-group_idM6.COVID

Creating single cell experiment object...

Aggregating count data...

Running differential state analysis...



  |======================================================================| 100%



ERROR: Error in pbDS(pb, method = pbDS.method, design = design, contrast = contrast, : Specified filtering options result in no genes in any clusters being tested. To force testing, consider modifying arguments 'min_cells' and/or 'filter'. See '?pbDS' for details.


In [13]:
subset_merged <- subset(merged, subset = Disease.Status3 %in% c('M12.COVID', 'MISC.recovered'))
M12_COVID_deg <- RunPseudobulkAnalysis(object = subset_merged,
                                    ident.var = "main_cluster_annotation",
                                    group.var = "Disease.Status3",
                                    sample.var = "sample.id2",
                                    group.1 = "MISC.recovered",
                                    group.2 = "M12.COVID")

Checking validity of names...

Converting Disease.Status3 to factor

Converting sample.id2 to factor

Contrast created: group_idMISC.recovered-group_idM12.COVID

Creating single cell experiment object...

Aggregating count data...

Running differential state analysis...



  |======================================================================| 100%




Formatting output...

Pseudobulk-analysis completed!



In [12]:
subset_merged <- subset(merged, subset = Disease.Status3 %in% c('D28.COVID', 'Baseline.MIS.C'))
D28_COVID_vs_Baseline_MISC_deg <- RunPseudobulkAnalysis(object = subset_merged,
                                    ident.var = "main_cluster_annotation",
                                    group.var = "Disease.Status3",
                                    sample.var = "sample.id2",
                                    group.1 = "D28.COVID",
                                    group.2 = "Baseline.MIS.C")

Checking validity of names...

Converting Disease.Status3 to factor

Converting sample.id2 to factor

Contrast created: group_idD28.COVID-group_idBaseline.MIS.C

Creating single cell experiment object...

Aggregating count data...

Running differential state analysis...



  |======================================================================| 100%




Formatting output...

Pseudobulk-analysis completed!



# Save RDS

In [73]:
list(D28_COVID_vs_Baseline_MISC=D28_COVID_vs_Baseline_MISC_deg,
    Baseline_MISC=Baseline_MISC_deg,
    D28_MISC=D28_MISC_deg,
    Baseline_COVID=Baseline_COVID_deg,
    D28_COVID=D28_COVID_deg,
    M12_COVID=M12_COVID_deg) -> bulk_deg_list

In [24]:
saveRDS(bulk_deg_list, "/data/niecr/cheongj/misc/results_seurat/DGE/pseudobulk/psedobulk_hspc.rds")

In [77]:
library(openxlsx)
out_dir <- '/data/niecr/cheongj/misc/results_seurat/DGE/pseudobulk/hspc_files/'
for (name in names(bulk_deg_list)) {
  bulk_deg_list_res <- bulk_deg_list[[name]]$res$table[[1]]
  # Create a list to store sheets
  sheets_list <- list()
  # Loop through each cell type in bulk_deg_list_res
  for (i in names(bulk_deg_list_res)) {
    deg <- bulk_deg_list_res[[i]]
    sheets_list[[i]] <- deg
  }
  write.xlsx(sheets_list, file = paste0(out_dir, name, '_pseudobulk_hspc.xlsx'))
}

In [78]:
library(openxlsx)
out_dir <- '/data/niecr/cheongj/misc/results_seurat/DGE/pseudobulk/hspc_files/'
sheets_list <- list()
for (name in names(bulk_deg_list)) {
  sheets_list[[name]] <- bulk_deg_list[[name]]$res$table[[1]]$HSPC
  write.xlsx(sheets_list, file = paste0(out_dir,'merged_pseudobulk_hspc.xlsx'))
}

# GSEA

In [ ]:
library(clusterProfiler)
library(org.Hs.eg.db)
library(enrichplot)

In [11]:
gseaobj <- function(df){
  tryCatch(
    {
      genelist <- df$logFC
      names(genelist) <- df$gene
      genelist <- na.omit(genelist)
      genelist <- sort(genelist, decreasing = T)
      
      gse <- gseGO(geneList=genelist, 
                   ont ="BP", 
                   keyType = "SYMBOL", 
                   minGSSize = 10, 
                   maxGSSize = 500,  # 500 genes seem too general
                   pvalueCutoff = 0.05, 
                   verbose = TRUE, 
                   OrgDb = "org.Hs.eg.db")
      gse
    }, error = function(e) {print("error")})
}

gseoplot <- function(x) {
  tryCatch(
    {
      dotplot(x, showCategory = 10, split = ".sign", font.size = 7) + facet_grid(.~.sign)
    }, error = function(e) {print("error")})
}

In [12]:
library(readxl)
file_path <- "/data/niecr/cheongj/misc/results_seurat/DGE/pseudobulk/hspc_files/merged_pseudobulk_hspc.xlsx"

# Get the names of all sheets
sheet_names <- excel_sheets(file_path)

# Initialize an empty list to store sheet data
bulk_deg_list_res <- list()

# Loop through each sheet and read its data
for (sheet_name in sheet_names) {
  # Read the sheet into a data frame
  sheet_data <- read_excel(file_path, sheet = sheet_name)
  
  # Add the data frame to the list with sheet name as key
  bulk_deg_list_res[[sheet_name]] <- list(sheet_data)
}

In [23]:
comparisonlist <- names(bulk_deg_list_res)

gsea_obj_list <- list()
gsea_result_list <- list()
gsea_plot_list <- list()

for (comparison in comparisonlist){
  

  print(paste0("start gsea ", comparison))
  
  
  celltype_deglist <- bulk_deg_list_res[[comparison]]
  
  celltype_deglist_flt <- lapply(celltype_deglist, function(x) x %>% filter(p_adj.loc < 0.05))
  celltype_deglist_flt <- lapply(celltype_deglist, function(x) x[!grepl("MT-|RPS|RPL", x$gene),])
  
  celltype_deglist_flt_gsea <- lapply(celltype_deglist_flt, gseaobj)
  celltype_deglist_flt_gsea_result <- lapply(celltype_deglist_flt_gsea, function(x) {
  tryCatch({
    x@result
  }, error = function(e) {
    NULL
  })
})
  celltype_deglist_flt_gsea_plot <- lapply(celltype_deglist_flt_gsea, gseoplot)
  celltype_deglist_flt_gsea_plot <- lapply(celltype_deglist_flt_gsea_plot, function(plot) {
  plot + ggtitle(paste(comparison, 'HSPC', sep = " "))})
                                 
  celltypes <- names(celltype_deglist_flt_gsea)
  
  for (i in celltypes){
    
    print(paste0("start gsea plot on ", comparison, " in ", i))
    
    tryCatch(
      {
        
        celltype_deglist_flt_gsea_plot[[i]] <-  celltype_deglist_flt_gsea_plot[[i]]+
          ggtitle(paste(comparison, i, sep = " "))
      }, error = function(e) {NULL})
  }
  
  gsea_obj_list[[comparison]] <- celltype_deglist_flt_gsea
  gsea_result_list[[comparison]] <- celltype_deglist_flt_gsea_result
  gsea_plot_list[[comparison]] <- celltype_deglist_flt_gsea_plot

}

[1] "start gsea D28_COVID_vs_Baseline_MISC"


preparing geneSet collections...

GSEA analysis...

Warning message in fgseaMultilevel(pathways = pathways, stats = stats, minSize = minSize, :
“For some pathways, in reality P-values are less than 1e-10. You can set the `eps` argument to zero for better estimation.”
leading edge analysis...

done...



[1] "start gsea Baseline_MISC"


preparing geneSet collections...

GSEA analysis...

leading edge analysis...

done...



[1] "start gsea D28_MISC"


...

preparing geneSet collections...

GSEA analysis...

Warning message in fgseaMultilevel(pathways = pathways, stats = stats, minSize = minSize, :
“There were 125 pathways for which P-values were not calculated properly due to unbalanced (positive and negative) gene-level statistic values. For such pathways pval, padj, NES, log2err are set to NA. You can try to increase the value of the argument nPermSimple (for example set it nPermSimple = 10000)”
Warning message in fgseaMultilevel(pathways = pathways, stats = stats, minSize = minSize, :
“For some of the pathways the P-values were likely overestimated. For such pathways log2err is set to NA.”
leading edge analysis...

done...



[1] "start gsea D28_COVID"


preparing geneSet collections...

GSEA analysis...

Warning message in fgseaMultilevel(pathways = pathways, stats = stats, minSize = minSize, :
“For some pathways, in reality P-values are less than 1e-10. You can set the `eps` argument to zero for better estimation.”
leading edge analysis...

done...



[1] "start gsea M12_COVID"


preparing geneSet collections...

GSEA analysis...

Warning message in fgseaMultilevel(pathways = pathways, stats = stats, minSize = minSize, :
“For some pathways, in reality P-values are less than 1e-10. You can set the `eps` argument to zero for better estimation.”
leading edge analysis...

done...



In [98]:
out_dir <- '/data/niecr/cheongj/misc/results_seurat/DGE/pseudobulk/hspc_files/'
setwd(out_dir)
saveRDS(gsea_obj_list, "deg_hspc_gsea_obj_list.rds")
saveRDS(gsea_result_list, "deg_hspc_gsea_result_list.rds")
saveRDS(gsea_plot_list, "deg_hspc_gsea_plot_list.rds")

In [24]:
out_dir <- '/data/niecr/cheongj/misc/results_seurat/DGE/pseudobulk/hspc_files/'
setwd(out_dir)
pdf("hspc_pseudobulk_gsea_plot_pval0.05.pdf")
gsea_plot_list
dev.off()

$D28_COVID_vs_Baseline_MISC
$D28_COVID_vs_Baseline_MISC[[1]]


$Baseline_MISC
$Baseline_MISC[[1]]


$D28_MISC
$D28_MISC[[1]]


$Baseline_COVID
$Baseline_COVID[[1]]


$D28_COVID
$D28_COVID[[1]]


$M12_COVID
$M12_COVID[[1]]



png 
  2

In [ ]:
a <- gsea_result_list
sheets_list <- list()
for (i in names(a)){
    sheets_list[[i]] <- a[[i]]
}
write.xlsx(sheets_list,'merged_pseudobulk_gsea_pval0.05.xlsx')

In [7]:
out_dir <- '/data/niecr/cheongj/misc/results_seurat/DGE/pseudobulk/hspc_files/'
setwd(out_dir)

In [9]:
volcano_plotlist <- list()
for (set in names(bulk_deg_list_res)){
  print(set)
  
  df <- as.data.frame(do.call(cbind,bulk_deg_list_res[[set]]))
    
    #df <- deg[[celltype]]
    
    df$sig <- "non-significant"
df$sig[df$p_adj.loc < 0.05] <- "significant"

df$direction <- NA
df$direction[df$logFC > 0] <- "up" 
df$direction[df$logFC < 0] <- "down" 

df$label <- NA
df$label[df$logFC > 0.25 & df$p_adj.loc < 0.05] <- df$gene[df$logFC > 0.25 & df$p_adj.loc < 0.05]
df$label[df$logFC < -0.25 & df$p_adj.loc < 0.05] <- df$gene[df$logFC < -0.25 & df$p_adj.loc < 0.05]

plot <- ggplot(df, aes(x = logFC, y = -log(p_adj.loc), label = label))+
  geom_point()+
  geom_point(data = subset(df, sig =="significant"), mapping = aes(color = direction))+
  ylab("-log(adj.pval)")+
  xlab("logFC")+
  xlim(c(-7, 7))+
  geom_text_repel(data = subset(df, df$direction== "up") ,
                  size = 3,
                  max.overlaps = 10,
                  xlim = c(5,6),
                  hjust=0,
                  force= 0.1,
                  direction ="y",
                  segment.size = 0.2,
                  segment.color = "grey50")+
  geom_text_repel(data = subset(df, df$direction== "down" ) ,
                  size = 3,
                  max.overlaps = 10,
                  xlim = c(-6,-5),
                  hjust=0,
                  force= 0.1,
                  direction ="y",
                  segment.size = 0.2,
                  segment.color = "grey50")+
  theme_classic()+
  scale_color_manual(values =
                       rev(wes_palette(n=2, name="Darjeeling1", type = "discrete")))+
  ggtitle(paste0("DEG ",set," HSPC") )
    
  volcano_plotlist[[paste(set, 'HSPC', sep = "_")]] <- plot        
}

pdf("./pseudobulk_deg_volcano.pdf")
volcano_plotlist
dev.off()

[1] "D28_COVID_vs_Baseline_MISC"
[1] "Baseline_MISC"
[1] "D28_MISC"
[1] "Baseline_COVID"
[1] "D28_COVID"
[1] "M12_COVID"


Warning message:
“Removed 5183 rows containing missing values or values outside the scale range
(`geom_text_repel()`).”
Warning message:
“Removed 5524 rows containing missing values or values outside the scale range
(`geom_text_repel()`).”
Warning message:
“Removed 5390 rows containing missing values or values outside the scale range
(`geom_text_repel()`).”
Warning message:
“Removed 5018 rows containing missing values or values outside the scale range
(`geom_text_repel()`).”
Warning message:
“ggrepel: 39 unlabeled data points (too many overlaps). Consider increasing max.overlaps”
Warning message:
“ggrepel: 82 unlabeled data points (too many overlaps). Consider increasing max.overlaps”
Warning message:
“Removed 5526 rows containing missing values or values outside the scale range
(`geom_text_repel()`).”
Warning message:
“Removed 6409 rows containing missing values or values outside the scale range
(`geom_text_repel()`).”
Warning message:
“Removed 4 rows containing missing values or valu

$D28_COVID_vs_Baseline_MISC_HSPC

$Baseline_MISC_HSPC

$D28_MISC_HSPC

$Baseline_COVID_HSPC

$D28_COVID_HSPC

$M12_COVID_HSPC


png 
  2